# HW10

## Q1.

Implement a Key-Value Search true, which allows no duplicates, but rather, updates the value associated with the key. This will change how `insert` works. Inherit from the augmented tree:

`class KeyValueBinarySearchTree(AugmentedBinarySearchTree):`

- The constructor should look like this: `def __init__(self, key_value_tuple, parent=None):`. Pick the key and value out separately in the constructor, and initialize the super with just the key, setting an instance variable `self.value` to the value.
- insert wont duplicate any more, and `addLeftChild` and `addRightChild` will need to take the tuple in as they call the constructor for us.
- implement a `__getitem__`, `__setitem__`, and `__delitem__` so that you can use code like:

`btreekv['f']` for searching

`btreekv['f']=10` for inserting

In [1]:
#your code here 

#import BinaryTree, BinarySearchTree and AugmentedBinarySearchTree from lecture notes - 

import uuid
class BinaryTree:
    def __init__(self, data, parent=None):
        self.data = data
        self.parent = parent
        self.uuid= uuid.uuid4()
        self.left = None
        self.right = None    
            
    def addLeftChild(self, data): 
        n = self.__class__(data, self)
        self.left = n
        return n
        
    def addRightChild(self, data):
        n = self.__class__(data, self)
        self.right = n
        return n
        
    def hasLeftChild(self):
        return self.left is not None

    def hasRightChild(self):
        return self.right is not None

    def hasAnyChild(self):
        return self.hasRightChild() or self.hasLeftChild()

    def hasBothChildren(self):
        return self.hasRightChild() and self.hasLeftChild()
    
    def hasNoChildren(self):
        return not self.hasRightChild() and not self.hasLeftChild()
    
    def isLeftChild(self):
        return self.parent and self.parent.left == self

    def isRightChild(self):
        return self.parent and self.parent.right == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.right or self.left)
    
            
    def preorder(self):
        if self.isLeftChild():
            yield (self.parent, self, "left")
        elif self.isRightChild():
            yield (self.parent, self, "right")
        if self.hasLeftChild():
            for v in self.left.preorder():
                yield v
        if self.hasRightChild():
            for v in self.right.preorder():
                yield v
                
class BinarySearchTree(BinaryTree):
        
    def __init__(self, data, parent=None):
        super().__init__(data, parent)
        self.count = 1

    def _insert_hook(self):
        pass
            
    def insert(self, data):
        if data < self.data:
            if self.hasLeftChild():
                self.left.insert(data)
            else:
                self.addLeftChild(data)
                self._insert_hook()
        elif data > self.data:
            if self.hasRightChild():
                self.right.insert(data)
            else:
                self.addRightChild(data)
                self._insert_hook()
        else: #duplicate value
            self.count += 1
            self._insert_hook()
            
    def search(self, data):
        if self.data == data:
            return self
        elif data < self.data and self.left:
            return self.left.search(data)
        elif data > self.data and self.right:
            return self.right.search(data)
        else:
            return None
        
    def delete(self, data):        
        if self.isRoot() and self.hasNoChildren() and self.data==data:#deleting the whole tree
            self.root=None#todo call a destructor that signals GC it can reap
            #self._update_sizes(up=False) #really tree is gone
            self._remove_hook()
        elif self.hasAnyChild():
            noder = self.search(data)
            if noder:
                self._remove(noder)
            else:
                raise ValueError("No such data {} in tree".format(data))
        else:
            raise ValueError("No such data {} in tree".format(data))

    def _remove_hook(self, up=False, by=1):
        pass
    
    def _remove(self, node):
        if node.isLeaf():
            if node.isLeftChild():
                node.parent.left = None
            else:
                node.parent.right = None
            #node._update_sizes(up=False, by=node.count)
            node._remove_hook(by=node.count)
            del node
        elif node.hasBothChildren():
            s = node.successor()
            #successor is guaranteed to have right child only
            s.spliceOut()
            #s._update_sizes(up=False, by=s.count)
            s._remove_hook(by=s.count)
            #handled more generally above
            #s.right.parent = s.parent
            #s.parent.left = s.right
            node.data = s.data
            #diff = s.count - node.count            
            #node._update_sizes(by=diff)
            node._remove_hook(up=True, by = s.count - node.count)
            node.count = s.count
            del s #the node became the successor
        else: # one child case
            if node.hasLeftChild():
                if node.isLeftChild():
                    node.left.parent = node.parent
                    node.parent.left = node.left
                elif node.isRightChild():
                    node.left.parent = node.parent
                    node.parent.right = node.left
                else: #root
                    self.root = node.left
                #node._update_sizes(up=False, by=node.count)
                node._remove_hook(by=node.count)
                del node
            else:
                if node.isLeftChild():
                    node.right.parent = node.parent
                    node.parent.left = node.right
                elif node.isRightChild():
                    node.right.parent = node.parent
                    node.parent.right = node.right
                else: #root
                    self.root = node.right
                #node._update_sizes(up=False, by=node.count)
                node._remove_hook(by=node.count)
                del node
                    
    def findMin(self):
        minnode = self
        while minnode.hasLeftChild():
            minnode = minnode.left
        return minnode
    
    def findMax(self):
        maxnode = self
        while maxnode.hasRightChild():
            maxnode = maxnode.right
        return maxnode
    
    def successor(self):
        s = None
        if self.hasRightChild():
            s = self.right.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    s = self.parent
                else:
                    self.parent.right=None
                    s = self.parent.successor()
                    self.parent.right=self
        return s
    
    def predecessor(self):
        p=None
        if self.hasLeftChild():
            p = self.left.findMax()
        else:
            if self.parent:
                if self.isRightChild():
                    p = self.parent
                else:
                    self.parent.left = None
                    p = self.parent.predecessor()
                    self.parent.left = self
        return p
            
    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.left = None
            else:
                self.parent.right = None
        elif self.hasAnyChild():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.left = self.left
                else:
                    self.parent.right = self.left
                self.left.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.left = self.right
                else:
                    self.parent.right = self.right
                self.right.parent = self.parent
       

    #now implement various pythonic things
    
    def __iter__(self):
        if self is not None:
            if self.hasLeftChild():
                for node in self.left:
                    yield node
            for _ in range(self.count):
                yield self
            if self.hasRightChild():
                for node in self.right:
                    yield node
                    
    def __len__(self):#expensive O(n) version
        start=0
        for node in self:
            start += 1
        return start
    
    def __getitem__(self, i):
        return self.ithorder(i+1)
    
    def __contains__(self, data):
        return self.search(data) is not None
    
class AugmentedBinarySearchTree(BinarySearchTree):
        
    def __init__(self, data, parent=None):
        super().__init__(data, parent)
        self.size = 1
        
    def _update_sizes(self, up=True, by=1):
        if up:
            inc = by
        else:
            inc = -by
        self.size += inc
        curr = self
        while curr.parent is not None:
            curr.parent.size += inc
            curr = curr.parent
       
    def _insert_hook(self):#insert up, by 1
        self._update_sizes()
            
    def _remove_hook(self, up=False, by=1):
        self._update_sizes(up, by)
        
    
    def ithorder(self, i): #starts from 1
        if self.hasLeftChild():
            a = self.left.size
        else:
            a = 0
        dupes = self.count - 1
        if  a+1 <= i  < a+1 + dupes:
            return self
        if i < a + 1 : #wont go here for size 0 on left
            return self.left.ithorder(i)
        elif  a+1 <= i  <= a+1 + dupes:
            return self
        else:#ok to have self.right here and not check right child
            return self.right.ithorder(i - a -1 -dupes)
       
    def _rankof(self, data):
        if self.data == data:#found at top
            if self.hasLeftChild():
                return self.left.size + self.count, self.count
            else:
                return self.count, self.count
        elif data < self.data and self.left:
            return self.left._rankof(data)
        elif data > self.data and self.right:
            rtup = self.right._rankof(data)
            if self.hasLeftChild():
                return self.count + self.left.size+rtup[0], rtup[1]
            else:
                return self.count + rtup[0], rtup[1]
        else:
            raise ValueError("{} not found".format(x))
            
    def rankof(self, data):
        ranktup = self._rankof(data)
        return [ranktup[0] - e for e in range(ranktup[1])]
    
    
    #now implement various pythonic things
    
    def __len__(self):
        return self.size

    
    def __getitem__(self, i):
        return self.ithorder(i+1)

In [2]:
#your code here

class KeyValueBinarySearchTree(AugmentedBinarySearchTree):
    """  
    This class implements a Key-Value Search which allows no duplicates, but rather, updates the 
    value associated with the key. It inherits from the AugmentedBinarySearchTree class.

    Examples
    --------
    >>> btreekv = KeyValueBinarySearchTree(('f', 1))
    >>> btreekv['f']
    1
    >>> btreekv['f'] = 42
    >>> btreekv['f']
    42
    >>> btreekv.__delitem__('g')
    Traceback (most recent call last):
    ...
    KeyError: 'The key - g - was not found.'
    """
    def __init__(self, key_value_tuple, parent=None):
        """
        Constructor to pick the key and value out separately and initialize the super with just the key,
         setting an instance variable self.value to the value.
        
        Parameters
        ----------
        key_value_tuple : tuple 
            tuple passed in that is used
        parent : parent class
            default to None, allow for different parent class implementation
        """ 
        # get the key and value from the provided tuple
        key, value = key_value_tuple
        
        # initialize the parent class
        super().__init__(key, parent)
        
        # set the instance variable for the value
        self.value = value
        
    def insert(self, key_value_tuple):
        """
        Insert a key value pair into the tree. If the key already exists, replace the value.
        
        Parameters
        ----------
        key_value_tuple : tuple 
            tuple passed in that is used for the insert
            
        WARNINGS: No duplicate keys allowed. New inserts for existing keys update the values for 
            that key, value pair.
        """
        key,value = key_value_tuple
        
        # use the key to find the position to make the insert
        # addLeftChild and addRightChild take the tuple in as they call the constructor for us. 
        
        # the key is larger, make the insert down the right branch
        if key > self.data:
            # right child already exists, add it there
            if self.hasRightChild():
                self.right.insert(key_value_tuple)
            else:
                # create new right child
                self.addRightChild(key_value_tuple)
                self._insert_hook()
                
        # the key is smaller, make the insert down the left branch    
        elif key < self.data:
            # left child already exists, add it there 
            if self.hasLeftChild():
                self.left.insert(key_value_tuple)
            else:
                # make new left child
                self.addLeftChild(key_value_tuple)
                self._insert_hook()
                
        # make the insert without right or left tree traversal    
        else: 
            self.value = value
            self._insert_hook()
        
    def __getitem__(self, key):
        """
        Get the value associated with the provided key.
    
        Parameters
        ----------
        key : string 
            the key to use for the search.
            
        Returns
        -------
        value : int
            the value found at index of the provided key.
            
        WARNINGS: Returns None if the key, value pair does not exist.
        
        """
        return self.search(key).value
    
    def __setitem__(self, key, value):
        """
        Set the value for the provided key.
    
        Parameters
        ----------
        key : string 
            the key to use for the set operation. 
        value : int 
            the value to use for the set operation. 
            
        Notes
        -----
        POST: The existing value is replaced if key is already present.  
        """        
        self.insert((key, value))
    
    def __delitem__(self, data):
        """
        Delete the (key, value) pair associated with the provided data argument.
    
        Parameters
        ----------
        data : string 
            the key to use for search to delete. 
            
        Raises
        ------
        KeyError : raised if the key provided is not in the tree.  
        """
        # the tree has only one key and it is the key that was requested to be deleted
        if self.isRoot() and self.hasNoChildren() and self.data == data:
            # empty the tree
            self.root = None
            self._remove_hook()
            
        # the three has more than one key, find the key to be deleted and execute the delete
        elif self.hasAnyChild(): 
            key_to_delete = self.search(data)
            
            # the key was found, do the delete
            if key_to_delete:
                self.__delitem__(key_to_delete)
            # the key was not found, raise a KeyError
            else:
                raise KeyError("The key - {} - was not found.".format(data))
                
        # the key was not found, raise a KeyError
        else:
            raise KeyError("The key - {} - was not found.".format(data))

In [3]:
# use doctest to run the docstring examples which demostrate the
# general use of the KeyValueBinarySearchTree class
from doctest import run_docstring_examples as dtest
dtest(KeyValueBinarySearchTree, globals(), verbose=True, name='KeyValueBinarySearchTree')

Finding tests in KeyValueBinarySearchTree
Trying:
    btreekv = KeyValueBinarySearchTree(('f', 1))
Expecting nothing
ok
Trying:
    btreekv['f']
Expecting:
    1
ok
Trying:
    btreekv['f'] = 42
Expecting nothing
ok
Trying:
    btreekv['f']
Expecting:
    42
ok
Trying:
    btreekv.__delitem__('g')
Expecting:
    Traceback (most recent call last):
    ...
    KeyError: 'The key - g - was not found.'
ok


It should all make sense....

In [4]:
btreekv = KeyValueBinarySearchTree(('f', 3))

In [5]:
kvdata=zip(list('jeihrifhkdfks'), range(13))

In [6]:
for k,v in kvdata:
    btreekv[k]=v

In [7]:
[(e.data, e.value) for e in list(btreekv)]

[('d', 9),
 ('e', 1),
 ('f', 10),
 ('h', 7),
 ('i', 5),
 ('j', 0),
 ('k', 11),
 ('r', 4),
 ('s', 12)]

In [8]:
btreekv['f']

10